In [17]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F

In [18]:
# 继承nn.Module 
class Net(nn.Module):
    def __init__(self):
        # 下式等价于nn.Module._init__(self)
        super(Net,self).__init__()
        # 1：输入图片单通道；6：输出通道数；5：卷积核5*5
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # 全连接层
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        #卷积 激活 池化
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        # reshape,"-1"表示自适应
        x = x.view(x.size()[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

params = list(net.parameters())
print(len(params))

for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [21]:

# nn.Conv2d输入必须是四维的 nSamples*nChannels*Height*Width 
input = t.randn(1, 1, 32, 32)
# print(input)
out = net(input)
print(out)
net.zero_grad()  # 将所有参数的梯度清零
out.backward(t.randn(1,10))

tensor([[-0.0725, -0.1008, -0.0320,  0.0781, -0.1165, -0.0096,  0.0502,
         -0.1304,  0.1236,  0.0992]])


In [23]:
output = net(input)
target = t.arange(1,11)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(38.3650)


In [24]:
net.zero_grad()
print("反向传播之前conv1.bias的梯度")
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前conv1.bias的梯度
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
反向传播之后conv1.bias的梯度
tensor(1.00000e-02 *
       [ 1.5142,  4.5706, -0.7846, -0.9049,  4.9241, -3.9225])


In [26]:
# 优化器
# weight = weight -learning_rate*gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)


In [27]:

import torch.optim as optim
# 新建一个优化器，指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(),lr=0.01)
# 在训练过程中，先梯度清零
optimizer.zero_grad()
output = net(input)
# 计算损失
loss = criterion(output,target)
# 反向传播
loss.backward()
# 更新参数
optimizer.step()